In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [42]:
class NeuralNetwork(object):
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        """NeuralNetwork Constructor.
            @self: means an object scope method
            @input_nodes: amount of input nodes (it is the input_array's last dimension)
            @hidden_nodes: an int amount of neurons nodes that process the data
            @output_nodes: an int amount of neurons nodes that process the data & retrieve the result
            @learning_rate: the rate/size of the learning/training process (it's as lower as good)
        """
        
        # Set number of nodes in input, hidden and output layers attrs.
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes

        # Initialize the weights from input to hidden prop
        # it's creating a matrix of numbers between 0.0 and HIDDEN amount ** -0.5, like 35 will be 0.166...
        # the last param is setting the dimensions using HIDDEN amount as lines & INPUT amount as columns
        self.weights_input_to_hidden = np.random.normal(0.0, self.hidden_nodes ** -0.5,
                                                        (self.hidden_nodes, self.input_nodes))

        # Initialize the weights from hidden to output prop
        # it's creating a matrix of numbers between 0.0 and OUTPUT amount ** -0.5, like 35 will be 0.166...
        # the last param is setting the dimensions using OUPUT amount as lines & HIDDEN amount as columns        
        self.weights_hidden_to_output = np.random.normal(0.0, self.output_nodes ** -0.5,
                                                         (self.output_nodes, self.hidden_nodes))
        
        # set the learning rate hyperparam
        self.lr = learning_rate

        # the activation function is the neuron work
        self.activation_function = lambda x: 1 / (1 + np.exp(-x))
        # the activation prime is the deviration of activation and it is known as "transfer function"
        self.activation_function_prime = lambda x: x*1

    def train(self, inputs_list, targets_list):
        """neuralNetworkObj.train method.
            It method trains the NeuralNetwork to know the RESULTS (targets) based on received EVIDENCES (inputs)
            @self: means an object scope method
            @inputs_list: the array containing the inputs (EVIDENCES)
            @targets_list: the array containing the results produced by the inputs
        """        
        
        # Convert inputs list to 2d array
#         inputs = np.array(inputs_list, ndmin=2).T
#         targets = np.array(targets_list, ndmin=2).T

        #### Implement the forward pass ####
        # weights is (n x input_dim), (a set of weights for each hidden node)
        # hidden inputs linear combination & generating activated output
        hidden_inputs = np.dot(self.weights_input_to_hidden, inputs.T)  # INPUT Layer generated hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)  # HIDDEN Layer prepared for output layer

        # values on output layer (end nodes), same steps before (linear_comb+activation) using out_refs
        final_inputs = np.dot(self.weights_hidden_to_output, hidden_outputs)  # HIDDEN Layer generated output entries
        final_outputs = self.activation_function_prime(final_inputs) # the output receive it entries derivated

        #### Implement the backward pass ####
        # (y - y^)Output layer error is the difference between desired target and actual output
        # error_term: y-y^(chapel)*ƒ’(h), ƒ’(h)=1, so error = error_term
        output_errors = targets_list - final_outputs.T

        # Backpropagated error
        hidden_errors = output_errors * self.weights_hidden_to_output.T  # errors propagated from output to hidden
        hidden_grad = hidden_outputs * (1 - hidden_outputs)  # hidden layer gradients from the output

        # wi = wi(current_weight) + n(learning-rate)*error-term*xi(current_input) --> epoch(term) error
        error_term = (hidden_errors * hidden_grad)
        # propagte input to hidden
        self.weights_input_to_hidden += self.lr * np.dot(error_term, inputs.T)        
        # propagte hidden to output
        self.weights_hidden_to_output += self.lr * output_errors * hidden_outputs.T 
        
        
    def run(self, inputs_list):
        # Run a forward pass through the network
        inputs = np.array(inputs_list, ndmin=2).T

        #### Implement the forward pass here ####
        # same from training function,but now using trained weights
        hidden_inputs = np.dot(self.weights_input_to_hidden, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)

        # linear combination out_weights & hidden_out
        final_inputs = np.dot(self.weights_hidden_to_output, hidden_outputs) 
        # it will return final_inputs
        final_outputs = self.activation_function_prime(final_inputs)

        return final_outputs

In [43]:
def MSE(y, Y):
    return np.mean((y-Y)**2)

In [44]:
import sys

### Set the hyperparameters here ###
epochs = 300
learning_rate = 0.05
hidden_nodes = 4 # Neurons
output_nodes = 1

inputs = np.array([[0,0], [0,1], [1,0], [1,1]])
train_features = np.array([0, 1, 1, 0])
val_features = np.array([0, 1, 1, 0])

N_i = inputs.shape[1]
network = NeuralNetwork(N_i, hidden_nodes, output_nodes, learning_rate)

losses = {'train':[], 'validation':[]}
for e in range(epochs):
    # Go through a random batch of 4 records from the training data set
    network.train(train_features, inputs)
    
    # Printing out the training progress
    train_loss = MSE(network.run(train_features), train_features.values)
    val_loss = MSE(network.run(val_features), val_features.values)
    sys.stdout.write("\rProgress: " + str(100 * e/float(epochs))[:4] \
                     + "% ... Training loss: " + str(train_loss)[:5] \
                     + " ... Validation loss: " + str(val_loss)[:5])
    
    losses['train'].append(train_loss)
    losses['validation'].append(val_loss)

ValueError: operands could not be broadcast together with shapes (4,2) (4,4) 